<a href="https://colab.research.google.com/github/pawel-wyszomirski/fb-campaign-downloader/blob/main/fb_campaign_downloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# Sprawdź sekrety
from google.colab import userdata
print("AP_ID:", "dostępne" if userdata.get('AP_ID') else "brak")
print("AP_SECRET:", "dostępne" if userdata.get('AP_SECRET') else "brak")
print("FACEBOOK_ACCESS_TOKEN:", "dostępne" if userdata.get('FACEBOOK_ACCESS_TOKEN') else "brak")
print("FACEBOOK_ACCOUNT_ID:", "dostępne" if userdata.get('FACEBOOK_ACCOUNT_ID') else "brak")

from facebook_business.api import FacebookAdsApi
from facebook_business.adobjects.adaccount import AdAccount
from google.colab import userdata

def test_connection():
    try:
        # Inicjalizacja API
        app_id = userdata.get('AP_ID')
        app_secret = userdata.get('AP_SECRET')
        access_token = userdata.get('FACEBOOK_ACCESS_TOKEN')
        account_id = userdata.get('FACEBOOK_ACCOUNT_ID').replace('act_', '')

        FacebookAdsApi.init(app_id, app_secret, access_token)

        # Próba połączenia z kontem
        account = AdAccount(f'act_{account_id}')
        account_details = account.api_get(fields=['name', 'id'])

        print("Połączenie udane!")
        print(f"Nazwa konta: {account_details.get('name')}")
        print(f"ID konta: {account_details.get('id')}")
        return True

    except Exception as e:
        print(f"Błąd połączenia: {str(e)}")
        print("\nSprawdź czy:")
        print("1. Token dostępu jest aktywny i ma odpowiednie uprawnienia")
        print("2. ID aplikacji i secret są poprawne")
        print("3. ID konta reklamowego jest poprawne")
        return False

# Test połączenia
test_connection()

AP_ID: dostępne
AP_SECRET: dostępne
FACEBOOK_ACCESS_TOKEN: dostępne
FACEBOOK_ACCOUNT_ID: dostępne
Połączenie udane!
Nazwa konta: kurs.lifeAI.guru
ID konta: act_489112560281560


True

In [16]:
# Instalacja wymaganych bibliotek
!pip install facebook_business pandas psutil

# Importy
import os
import glob
import psutil
from datetime import datetime, timedelta
from facebook_business.api import FacebookAdsApi
from facebook_business.adobjects.adaccount import AdAccount
from facebook_business.adobjects.campaign import Campaign
from facebook_business.adobjects.ad import Ad
from facebook_business.adobjects.adcreative import AdCreative
import pandas as pd
from google.colab import userdata

# Uruchomienie programu
if __name__ == "__main__":
    print("\nSprawdzanie środowiska przed uruchomieniem...")

    # Sprawdzanie wymaganych bibliotek
    required_packages = ['facebook_business', 'pandas', 'datetime', 'psutil']
    missing_packages = []

    for package in required_packages:
        try:
            __import__(package)
            print(f"✓ Biblioteka {package} jest dostępna")
        except ImportError:
            missing_packages.append(package)
            print(f"✗ Brak biblioteki {package}")

    # Sprawdzanie dostępu do API Facebooka
    print("\nSprawdzanie dostępu do API Facebooka:")
    api_secrets = {
        'AP_ID': userdata.get('AP_ID'),
        'AP_SECRET': userdata.get('AP_SECRET'),
        'FACEBOOK_ACCESS_TOKEN': userdata.get('FACEBOOK_ACCESS_TOKEN'),
        'FACEBOOK_ACCOUNT_ID': userdata.get('FACEBOOK_ACCOUNT_ID')
    }

    missing_secrets = [key for key, value in api_secrets.items() if not value]

    if missing_secrets:
        print("✗ Brakujące sekrety API:")
        for secret in missing_secrets:
            print(f"  - {secret}")
    else:
        print("✓ Wszystkie wymagane sekrety API są dostępne")

    # Sprawdzanie uprawnień zapisu
    try:
        test_file = 'test_write_permission.txt'
        with open(test_file, 'w') as f:
            f.write('test')
        os.remove(test_file)
        print("✓ Uprawnienia zapisu do katalogu są poprawne")
    except Exception as e:
        print(f"✗ Problem z uprawnieniami zapisu: {str(e)}")

    # Sprawdzanie istniejących plików CSV
    csv_files = glob.glob('facebook_ads_data_*.csv')
    if csv_files:
        print(f"\nZnaleziono {len(csv_files)} istniejących plików z danymi:")
        for file in csv_files:
            file_size = os.path.getsize(file)
            file_time = datetime.fromtimestamp(os.path.getmtime(file))
            print(f"- {file}")
            print(f"  Rozmiar: {file_size/1024:.2f} KB")
            print(f"  Ostatnia modyfikacja: {file_time}")
    else:
        print("\nNie znaleziono wcześniejszych plików z danymi")

    # Sprawdzanie dostępnej pamięci
    memory = psutil.virtual_memory()
    print(f"\nDostępna pamięć: {memory.available/1024/1024:.0f} MB")
    print(f"Użycie pamięci: {memory.percent}%")

    # Sprawdzanie, czy wszystko jest gotowe do uruchomienia
    if missing_packages:
        print("\n⚠️ UWAGA: Brakuje wymaganych bibliotek!")
        print("Zainstaluj brakujące biblioteki komendą:")
        print(f"pip install {' '.join(missing_packages)}")
    elif missing_secrets:
        print("\n⚠️ UWAGA: Brakuje wymaganych sekretów API!")
        print("Upewnij się, że wszystkie sekrety są dostępne w Colab")
    else:
        print("\n✓ Wszystkie wymagania spełnione")
        print("Uruchamianie programu głównego...\n")
        main()

def test_connection():
    """
    Testuje połączenie z Facebook API
    """
    try:
        app_id = userdata.get('AP_ID')
        app_secret = userdata.get('AP_SECRET')
        access_token = userdata.get('FACEBOOK_ACCESS_TOKEN')
        account_id = userdata.get('FACEBOOK_ACCOUNT_ID').replace('act_', '')

        print("\nSprawdzanie konfiguracji:")
        print(f"ID konta: {account_id}")

        FacebookAdsApi.init(app_id, app_secret, access_token)

        account = AdAccount(f'act_{account_id}')
        account_details = account.api_get(fields=['name', 'id', 'currency'])

        print("\nPołączenie udane!")
        print(f"Nazwa konta: {account_details.get('name')}")
        print(f"ID konta: {account_details.get('id')}")
        print(f"Waluta: {account_details.get('currency')}")
        return account_id

    except Exception as e:
        print(f"\nBłąd połączenia: {str(e)}")
        return None

def get_campaigns_list(account_id):
    """
    Pobiera listę dostępnych kampanii
    """
    try:
        account = AdAccount(f'act_{account_id}')
        campaigns = account.get_campaigns(fields=['name', 'id', 'status'])

        campaigns_list = []
        print("\nDostępne kampanie:")
        for i, campaign in enumerate(campaigns, 1):
            print(f"{i}. {campaign['name']} (ID: {campaign['id']}) - Status: {campaign['status']}")
            campaigns_list.append({
                'index': i,
                'id': campaign['id'],
                'name': campaign['name'],
                'status': campaign['status']
            })
        return campaigns_list
    except Exception as e:
        print(f"Błąd podczas pobierania listy kampanii: {str(e)}")
        return None

def get_campaign_insights(campaign, start_date, end_date):
    """
    Pobiera statystyki kampanii za wybrany okres
    """
    try:
        insights_params = {
            'time_range': {'since': start_date, 'until': end_date},
            'time_increment': 1,
            'fields': [
                'impressions',
                'clicks',
                'spend',
                'reach',
                'frequency',
                'cpc',
                'cpm',
                'ctr'
            ]
        }

        insights = campaign.get_insights(params=insights_params)
        print(f"Pobrano {len(insights) if insights else 0} dni danych dla kampanii {campaign['name']}")
        return insights
    except Exception as e:
        print(f"Błąd podczas pobierania statystyk kampanii {campaign.get('name', '')}: {str(e)}")
        return []

def validate_date(date_str):
    """
    Sprawdza poprawność formatu daty
    """
    try:
        datetime.strptime(date_str, '%Y-%m-%d')
        return True
    except ValueError:
        return False

def get_date_input(prompt):
    """
    Pobiera i waliduje datę od użytkownika
    """
    while True:
        date_str = input(prompt + " (format YYYY-MM-DD): ")
        if validate_date(date_str):
            return date_str
        print("Nieprawidłowy format daty. Użyj formatu YYYY-MM-DD (np. 2024-01-01)")
def get_detailed_campaign_data(account_id, start_date, end_date, campaign_id=None):
    try:
        account = AdAccount(f'act_{account_id}')
        print(f"\nPobieranie danych dla okresu: {start_date} - {end_date}")

        # Rozszerzone pola dla kampanii
        campaign_fields = [
            'name',
            'objective',
            'status',
            'start_time',
            'budget_remaining',
            'daily_budget',
            'lifetime_budget',
            'bid_strategy',
            'special_ad_categories'
        ]

        params = {'fields': campaign_fields}
        if campaign_id:
            params['filtering'] = [{'field': 'id', 'operator': 'EQUAL', 'value': campaign_id}]

        campaigns = account.get_campaigns(params=params)
        all_data = []
        campaign_count = 0

        for campaign in campaigns:
            campaign_count += 1
            print(f"\nPrzetwarzanie kampanii {campaign_count}: {campaign['name']}")

            # Rozszerzone pola dla statystyk
            insights_params = {
                'time_range': {'since': start_date, 'until': end_date},
                'time_increment': 1,
                'fields': [
                    'impressions',
                    'clicks',
                    'spend',
                    'reach',
                    'frequency',
                    'cpc',
                    'cpm',
                    'ctr',
                    'actions',
                    'cost_per_action_type',
                    'website_ctr',
                    'unique_clicks',
                    'unique_ctr',
                    'cost_per_unique_click',
                    'outbound_clicks',
                    'cost_per_outbound_click'
                ]
            }

            # Poprawione pola dla reklam - usunięto niedostępne pola
            ads = campaign.get_ads(fields=[
                'name',
                'status',
                'creative',
                'effective_status',
                'configured_status',
                'bid_amount',
                'bid_info',
                'targeting',
                'tracking_specs'
            ])

            ads_list = list(ads)
            print(f"Znaleziono {len(ads_list)} reklam w kampanii")

            for ad in ads_list:
                try:
                    print(f"Przetwarzanie reklamy: {ad['name']}")

                    # W funkcji get_detailed_campaign_data, zmień sekcję pobierania danych kreacji:

                    # Rozszerzone pobieranie danych kreacji
                    creative = None
                    creative_details = {}
                    if 'creative' in ad and 'id' in ad['creative']:
                        creative = AdCreative(ad['creative']['id'])
                        # Usunięto problematyczne pola i pozostawiono tylko te dostępne
                        creative_fields = [
                            'body',
                            'title',
                            'image_url',
                            'video_id',
                            'link_url',
                            'thumbnail_url',
                            'object_story_spec',
                            'call_to_action_type'
                        ]
                        try:
                            creative_details = creative.api_get(fields=creative_fields)
                            print(f"Pobrano dane kreacji dla reklamy: {ad['name']}")

                            # Pobieranie dodatkowych informacji z object_story_spec
                            if 'object_story_spec' in creative_details:
                                story_spec = creative_details['object_story_spec']
                                if isinstance(story_spec, dict):
                                    if 'link_data' in story_spec:
                                        creative_details['link_description'] = story_spec['link_data'].get('message', '')
                                        creative_details['link_title'] = story_spec['link_data'].get('name', '')
                                    if 'video_data' in story_spec:
                                        creative_details['video_description'] = story_spec['video_data'].get('message', '')
                                        creative_details['video_title'] = story_spec['video_data'].get('title', '')

                            # Dodanie URL podglądu
                            try:
                                previews = creative.get_previews(params={
                                    'ad_format': 'DESKTOP_FEED_STANDARD'
                                })
                                if previews:
                                    creative_details['preview_url'] = previews[0].get('body', '')
                            except Exception as e:
                                print(f"Błąd podczas pobierania podglądu: {str(e)}")
                                creative_details['preview_url'] = ''

                        except Exception as e:
                            print(f"Błąd podczas pobierania szczegółów kreacji: {str(e)}")
                            creative_details = {}

                        # Pobieranie podglądu reklamy
                        try:
                            preview_params = {
                                'ad_format': 'DESKTOP_FEED_STANDARD'
                            }
                            previews = creative.get_previews(params=preview_params)
                            if previews:
                                creative_details['preview_url'] = previews[0].get('body', '')
                        except Exception as e:
                            print(f"Błąd podczas pobierania podglądu: {str(e)}")

                    # Pobieranie statystyk dla reklamy
                    ad_insights = ad.get_insights(params=insights_params)

                    for insight in ad_insights:
                        # Przetwarzanie danych o konwersjach
                        actions = {action['action_type']: action['value']
                                 for action in insight.get('actions', [])}

                        action_costs = {cost['action_type']: cost['value']
                                      for cost in insight.get('cost_per_action_type', [])}

                    ad_data = {
                        # Dane kampanii
                        'campaign_id': campaign['id'],
                        'campaign_name': campaign['name'],
                        'campaign_objective': campaign.get('objective'),
                        'campaign_status': campaign['status'],
                        'daily_budget': campaign.get('daily_budget'),
                        'lifetime_budget': campaign.get('lifetime_budget'),
                        'bid_strategy': campaign.get('bid_strategy'),

                        # Dane reklamy
                        'ad_id': ad['id'],
                        'ad_name': ad['name'],
                        'ad_status': ad['status'],
                        'effective_status': ad.get('effective_status'),
                        'bid_amount': ad.get('bid_amount'),

                        # Dane kreacji
                        'ad_text': creative_details.get('body', ''),
                        'ad_title': creative_details.get('title', ''),
                        'image_url': creative_details.get('image_url', ''),
                        'video_id': creative_details.get('video_id', ''),
                        'link_url': creative_details.get('link_url', ''),
                        'preview_url': creative_details.get('preview_url', ''),
                        'call_to_action_type': creative_details.get('call_to_action_type', ''),
                        'link_description': creative_details.get('link_description', ''),
                        'link_title': creative_details.get('link_title', ''),
                        'video_description': creative_details.get('video_description', ''),
                        'video_title': creative_details.get('video_title', ''),

                            # Dane statystyczne
                            'date': insight.get('date_start'),
                            'impressions': insight.get('impressions', 0),
                            'clicks': insight.get('clicks', 0),
                            'unique_clicks': insight.get('unique_clicks', 0),
                            'spend': insight.get('spend', 0),
                            'reach': insight.get('reach', 0),
                            'frequency': insight.get('frequency', 0),
                            'cpc': insight.get('cpc', 0),
                            'cpm': insight.get('cpm', 0),
                            'ctr': insight.get('ctr', 0),
                            'unique_ctr': insight.get('unique_ctr', 0),
                            'cost_per_unique_click': insight.get('cost_per_unique_click', 0),

                            # Dane o konwersjach
                            'link_clicks': actions.get('link_click', 0),
                            'page_engagement': actions.get('page_engagement', 0),
                            'post_engagement': actions.get('post_engagement', 0),
                            'purchases': actions.get('purchase', 0),
                            'adds_to_cart': actions.get('add_to_cart', 0),
                            'leads': actions.get('lead', 0),

                            # Koszty konwersji
                            'cost_per_link_click': action_costs.get('link_click', 0),
                            'cost_per_purchase': action_costs.get('purchase', 0),
                            'cost_per_lead': action_costs.get('lead', 0)
                        }

                    all_data.append(ad_data)

                except Exception as e:
                    print(f"Błąd podczas przetwarzania reklamy {ad.get('name', '')}: {str(e)}")
                    continue
                # Po zebraniu wszystkich danych w all_data
        if not all_data:
            print("\nNie znaleziono żadnych danych do pobrania")
            return None

        print(f"\nZebrano {len(all_data)} rekordów z danymi")

        try:
            # Tworzenie DataFrame
            print("Tworzenie DataFrame...")
            df = pd.DataFrame(all_data)
            print(f"Utworzono DataFrame z {len(df)} wierszami i {len(df.columns)} kolumnami")

            # Formatowanie kolumn numerycznych
            numeric_columns = [
                'impressions', 'clicks', 'unique_clicks', 'spend', 'reach',
                'frequency', 'cpc', 'cpm', 'ctr', 'unique_ctr',
                'cost_per_unique_click', 'link_clicks', 'page_engagement',
                'post_engagement', 'purchases', 'adds_to_cart', 'leads',
                'cost_per_link_click', 'cost_per_purchase', 'cost_per_lead'
            ]

            print("Formatowanie kolumn numerycznych...")
            for col in numeric_columns:
                if col in df.columns:
                    df[col] = pd.to_numeric(df[col], errors='coerce')

            # Zapis do CSV
            filename = f'facebook_ads_data_{start_date}_to_{end_date}_detailed.csv'
            print(f"\nZapisywanie danych do pliku: {filename}")

            # Dodajmy pełną ścieżkę do pliku
            import os
            full_path = os.path.join(os.getcwd(), filename)

            # Zapis z dodatkowymi parametrami bezpieczeństwa
            df.to_csv(full_path,
                     index=False,
                     encoding='utf-8-sig',  # używamy utf-8-sig dla lepszej kompatybilności
                     date_format='%Y-%m-%d',
                     float_format='%.2f')

            # Sprawdzenie czy plik został utworzony
            if os.path.exists(full_path):
                print(f"Plik został pomyślnie zapisany w: {full_path}")
                print(f"Rozmiar pliku: {os.path.getsize(full_path)} bajtów")
            else:
                print("BŁĄD: Plik nie został utworzony!")

            # Wyświetlenie podsumowania
            print("\nPodsumowanie danych:")
            print(f"Liczba kampanii: {campaign_count}")
            print(f"Liczba rekordów: {len(df)}")

            # Podstawowe statystyki
            if len(df) > 0:
                print("\nPodstawowe statystyki:")
                print("\nSuma wydatków według kampanii:")
                spend_summary = df.groupby('campaign_name')['spend'].sum().round(2)
                print(spend_summary)

                print("\nŚredni CTR według kampanii:")
                ctr_summary = df.groupby('campaign_name')['ctr'].mean().round(2)
                print(ctr_summary)

            return df

        except Exception as e:
            print(f"\nWystąpił błąd podczas zapisywania danych: {str(e)}")
            # Dodatkowa diagnostyka błędu
            import traceback
            print("\nSzczegóły błędu:")
            print(traceback.format_exc())
            return None
        # Wyświetlenie rozszerzonego podsumowania
        print("\nPodsumowanie danych:")
        print(f"Liczba kampanii: {campaign_count}")
        print(f"Liczba rekordów: {len(df)}")

        # Podsumowanie według kampanii
        print("\nStatystyki według kampanii:")
        campaign_summary = df.groupby('campaign_name').agg({
            'spend': 'sum',
            'impressions': 'sum',
            'clicks': 'sum',
            'link_clicks': 'sum',
            'purchases': 'sum',
            'adds_to_cart': 'sum'
        }).round(2)

        campaign_summary['ctr'] = (campaign_summary['clicks'] / campaign_summary['impressions'] * 100).round(2)
        campaign_summary['cpc'] = (campaign_summary['spend'] / campaign_summary['clicks']).round(2)
        campaign_summary['conversion_rate'] = (campaign_summary['purchases'] / campaign_summary['link_clicks'] * 100).round(2)
        print(campaign_summary)

        # Podsumowanie według reklam
        print("\nTop 5 reklam według CTR:")
        ad_summary = df.groupby(['campaign_name', 'ad_name']).agg({
            'spend': 'sum',
            'impressions': 'sum',
            'clicks': 'sum',
            'link_clicks': 'sum',
            'purchases': 'sum',
            'adds_to_cart': 'sum'
        }).round(2)

        ad_summary['ctr'] = (ad_summary['clicks'] / ad_summary['impressions'] * 100).round(2)
        ad_summary['cpc'] = (ad_summary['spend'] / ad_summary['clicks']).round(2)
        ad_summary['conversion_rate'] = (ad_summary['purchases'] / ad_summary['link_clicks'] * 100).round(2)
        print(ad_summary.sort_values('ctr', ascending=False).head())

        # Trendy dzienne
        print("\nTrendy dzienne (ostatnie 7 dni):")
        daily_trends = df.groupby('date').agg({
            'spend': 'sum',
            'impressions': 'sum',
            'clicks': 'sum',
            'purchases': 'sum',
            'ctr': 'mean',
            'cpc': 'mean'
        }).round(2)
        print(daily_trends.tail(7))

        return df

    except Exception as e:
        print(f"\nWystąpił błąd: {str(e)}")
        return None

def main():
    print("Sprawdzanie połączenia z Facebook API...")
    account_id = test_connection()

    if account_id:
        while True:
            print("\nWybierz opcję:")
            print("1. Pobierz dane dla wszystkich kampanii")
            print("2. Pobierz dane dla wybranej kampanii")
            print("3. Analizuj istniejące dane")
            print("4. Zakończ")

            choice = input("\nTwój wybór (1-4): ")

            if choice == "4":
                print("Koniec programu")
                break

            if choice in ["1", "2"]:
                # Pobieranie dat
                while True:
                    start_date = get_date_input("\nPodaj datę początkową")
                    end_date = get_date_input("Podaj datę końcową")

                    # Sprawdzenie dat
                    if datetime.strptime(end_date, '%Y-%m-%d') >= datetime.strptime(start_date, '%Y-%m-%d'):
                        break
                    else:
                        print("Data końcowa musi być późniejsza niż początkowa!")

                campaign_id = None
                if choice == "2":
                    campaigns = get_campaigns_list(account_id)
                    if campaigns:
                        while True:
                            try:
                                campaign_index = int(input("\nWybierz numer kampanii: ")) - 1
                                if 0 <= campaign_index < len(campaigns):
                                    campaign_id = campaigns[campaign_index]['id']
                                    print(f"Wybrano kampanię: {campaigns[campaign_index]['name']}")
                                    break
                                else:
                                    print("Nieprawidłowy numer kampanii")
                            except ValueError:
                                print("Wprowadź poprawny numer")
                    else:
                        print("Nie udało się pobrać listy kampanii")
                        continue

                # Pobieranie danych
                df = get_detailed_campaign_data(
                    account_id=account_id,
                    start_date=start_date,
                    end_date=end_date,
                    campaign_id=campaign_id
                )

                if df is not None:
                    print("\nCzy chcesz pobrać dane dla innego okresu/kampanii? (t/n)")
                    if input().lower() != 't':
                        break

            elif choice == "3":
                import glob
                csv_files = glob.glob('facebook_ads_data_*_detailed.csv')
                if csv_files:
                    print("\nDostępne pliki z danymi:")
                    for i, file in enumerate(csv_files, 1):
                        print(f"{i}. {file}")

                    try:
                        file_index = int(input("\nWybierz numer pliku do analizy: ")) - 1
                        if 0 <= file_index < len(csv_files):
                            df = pd.read_csv(csv_files[file_index])

                            print("\nWybierz typ analizy:")
                            print("1. Podsumowanie kampanii")
                            print("2. Najlepsze reklamy")
                            print("3. Trendy dzienne")

                            analysis_choice = input("\nTwój wybór (1-3): ")

                            if analysis_choice == "1":
                                campaign_analysis(df)
                            elif analysis_choice == "2":
                                ad_analysis(df)
                            elif analysis_choice == "3":
                                daily_analysis(df)
                            else:
                                print("Nieprawidłowy wybór")
                        else:
                            print("Nieprawidłowy numer pliku")
                    except ValueError:
                        print("Wprowadź poprawny numer")
                else:
                    print("Brak dostępnych plików z danymi")
            else:
                print("Nieprawidłowy wybór")
    else:
        print("Nie udało się nawiązać połączenia z Facebook API. Sprawdź konfigurację.")

# Uruchomienie programu
if __name__ == "__main__":
    print("\nSprawdzanie środowiska przed uruchomieniem...")

    # Sprawdzanie wymaganych bibliotek
    required_packages = ['facebook_business', 'pandas', 'datetime']
    missing_packages = []

    for package in required_packages:
        try:
            __import__(package)
            print(f"✓ Biblioteka {package} jest dostępna")
        except ImportError:
            missing_packages.append(package)
            print(f"✗ Brak biblioteki {package}")

    # Sprawdzanie dostępu do API Facebooka
    print("\nSprawdzanie dostępu do API Facebooka:")
    api_secrets = {
        'AP_ID': userdata.get('AP_ID'),
        'AP_SECRET': userdata.get('AP_SECRET'),
        'FACEBOOK_ACCESS_TOKEN': userdata.get('FACEBOOK_ACCESS_TOKEN'),
        'FACEBOOK_ACCOUNT_ID': userdata.get('FACEBOOK_ACCOUNT_ID')
    }

    missing_secrets = [key for key, value in api_secrets.items() if not value]

    if missing_secrets:
        print("✗ Brakujące sekrety API:")
        for secret in missing_secrets:
            print(f"  - {secret}")
    else:
        print("✓ Wszystkie wymagane sekrety API są dostępne")

    # Sprawdzanie uprawnień zapisu
    import os
    try:
        test_file = 'test_write_permission.txt'
        with open(test_file, 'w') as f:
            f.write('test')
        os.remove(test_file)
        print("✓ Uprawnienia zapisu do katalogu są poprawne")
    except Exception as e:
        print(f"✗ Problem z uprawnieniami zapisu: {str(e)}")

    # Sprawdzanie istniejących plików CSV
    csv_files = glob.glob('facebook_ads_data_*.csv')
    if csv_files:
        print(f"\nZnaleziono {len(csv_files)} istniejących plików z danymi:")
        for file in csv_files:
            file_size = os.path.getsize(file)
            file_time = datetime.fromtimestamp(os.path.getmtime(file))
            print(f"- {file}")
            print(f"  Rozmiar: {file_size/1024:.2f} KB")
            print(f"  Ostatnia modyfikacja: {file_time}")
    else:
        print("\nNie znaleziono wcześniejszych plików z danymi")

    # Sprawdzanie dostępnej pamięci
    import psutil
    memory = psutil.virtual_memory()
    print(f"\nDostępna pamięć: {memory.available/1024/1024:.0f} MB")
    print(f"Użycie pamięci: {memory.percent}%")

    # Sprawdzanie, czy wszystko jest gotowe do uruchomienia
    if missing_packages:
        print("\n⚠️ UWAGA: Brakuje wymaganych bibliotek!")
        print("Zainstaluj brakujące biblioteki komendą:")
        print(f"pip install {' '.join(missing_packages)}")
    elif missing_secrets:
        print("\n⚠️ UWAGA: Brakuje wymaganych sekretów API!")
        print("Upewnij się, że wszystkie sekrety są dostępne w Colab")
    else:
        print("\n✓ Wszystkie wymagania spełnione")
        print("Uruchamianie programu głównego...\n")
        main()



Sprawdzanie środowiska przed uruchomieniem...
✓ Biblioteka facebook_business jest dostępna
✓ Biblioteka pandas jest dostępna
✓ Biblioteka datetime jest dostępna
✓ Biblioteka psutil jest dostępna

Sprawdzanie dostępu do API Facebooka:
✓ Wszystkie wymagane sekrety API są dostępne
✓ Uprawnienia zapisu do katalogu są poprawne

Nie znaleziono wcześniejszych plików z danymi

Dostępna pamięć: 11688 MB
Użycie pamięci: 9.9%

✓ Wszystkie wymagania spełnione
Uruchamianie programu głównego...

Sprawdzanie połączenia z Facebook API...

Sprawdzanie konfiguracji:
ID konta: 489112560281560

Połączenie udane!
Nazwa konta: kurs.lifeAI.guru
ID konta: act_489112560281560
Waluta: PLN

Wybierz opcję:
1. Pobierz dane dla wszystkich kampanii
2. Pobierz dane dla wybranej kampanii
3. Analizuj istniejące dane
4. Zakończ

Twój wybór (1-4): 2

Podaj datę początkową (format YYYY-MM-DD): 2024-07-01
Podaj datę końcową (format YYYY-MM-DD): 2024-11-11

Dostępne kampanie:
1. Zakup - Zbiorcza - Claude (ID: 1202145715230

NameError: name 'ad_analysis' is not defined